In [5]:
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.postprocessing import *
from library_dicom.dicom_processor.tools.folders import *
import os 
import csv

In [ ]:
folder = '/media/deeplearning/Elements/inference/RELEVANCE_NIFTI'
list_dir = os.listdir(folder)
liste_inference = []
for file_ in list_dir : 
    sub = []
    sub.append(os.path.join(folder, file_))
    #liste_inference.append(os.path.join(folder, file_))
    sub.append(file_.split('_')[0])
    #liste_study_uid.append(file_.split('_')[0])
    liste_inference.append(sub)

In [ ]:
csv_path= '/media/deeplearning/Elements/RELEVANCE_NIFTI/RELEVANCE_NIFTI_SCREENING_v3.csv'

with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    dataset = []
    for row in reader :
        dataset.append(row)
        
del dataset[0] #enlever première ligne

In [ ]:
#with study uid, find path pet 
for study in liste_inference : 
    for row in dataset : 
        if study[1] == row[1] : 
            study.append(row[-1]) #pet path 
            study.append(row[0]) #patient id

In [ ]:
remove_ = []
for study in liste_inference : 
    if len(study) != 4 : 
        remove_.append(study)

for r in remove_: 
    liste_inference.remove(r)

In [ ]:
print('Nombre de prédictions : ', len(liste_inference))

In [ ]:
mip_directory = '/media/deeplearning/Elements/RELEVANCE_MIP'
filenames = []
cpt =0
for study in liste_inference : 
   print(cpt)
   uid = study[1]
   sub = []
   inference_array = read_inference(study[0])
   pet_array, size = read_img(study[2])

   sub.append(mip_projection(pet_array, 90, mip_directory, uid, 'pet', cmap = 'gray', vmin= 0 , vmax = 6))
   sub.append(mip_superposition_show(inference_array, pet_array, 90, uid, vmin = 0, vmax = 6, cmap_pet = 'gray', cmap_mask = 'hsv', alpha = 0.6, save = True, directory = mip_directory))
   sub.append(uid)
   filenames.append(sub)
   cpt += 1

In [ ]:
err = '1.2.840.113704.1.111.4212.1406734680.46'
remove_ = []
for mip in filenames : 
    if err in mip : 
        remove_.append(mip)

for r in remove_ : 
    filenames.remove(r)

In [ ]:
write_json_file(mip_directory, 'filenames', filenames)

In [ ]:
output_path_name = mip_directory+'/'+'mip'
create_pdf_mip(filenames, output_path_name)

In [ ]:
# EXCLUDED MASK 

In [ ]:
liste_uid = ['1.2.840.113619.2.55.3.34213890.661.1394086511.913',
'1.3.46.670589.16.2.2.164.1.157.1.20140415.141816.4561358',
'1.3.51.0.1.1.10.49.10.222.2458375.2457056',
'2.16.56.465769650.2551326252.2240783396',
'2.16.56.465769650.2551326252.2606294451',
'1.2.276.0.7230010.3.8699790']

study = []
for uid in liste_uid : 
    for row in dataset : 
        if uid == row[1] : 
            study.append(row)

predictions = []
for uid in liste_uid : 
    for data in liste_inference : 
        if uid == data[1] : 
            predictions.append(data)     

In [ ]:
nifti_directory = '/media/deeplearning/Elements/inference'
filename = 'excluded_mask_pred.csv'


with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "INFERENCE"])
    for s, pred in zip(study, predictions) : 
        csv_writer.writerow([s[0], s[1], pred[0]])